In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.datasets import  load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from torch import nn


In [2]:
from accuracy_score import acc

In [67]:
data=load_wine()

X = data['data']

y = data['target']

stsc = StandardScaler()

X_std = stsc.fit_transform(X)

pca=PCA(n_components=2)

X_transformed = pca.fit_transform(X_std)

In [68]:
X_transformed = torch.tensor(X_transformed, dtype=torch.float32)

y = torch.tensor(y, dtype=torch.int64)

model = nn.Sequential(nn.Linear(2, 3), nn.Softmax(dim=1))

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

loss_fn = nn.CrossEntropyLoss()

In [69]:
dataloader = DataLoader(list(zip(X_transformed, y)), batch_size=10, shuffle=True)

In [80]:
epochs = 5

for epoch in range(epochs):
    for x, y in dataloader:
        optimizer.zero_grad()
        y_pred = model(x)
        accuraccy=acc.accuracy_score(y, torch.argmax((y_pred), dim=1))
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch + 1}, Loss: {loss.item():.3f}, Accuracy: {accuraccy:.2f}')

Epoch: 1, Loss: 0.600, Accuracy: 1.00
Epoch: 2, Loss: 0.552, Accuracy: 1.00
Epoch: 3, Loss: 0.591, Accuracy: 1.00
Epoch: 4, Loss: 0.554, Accuracy: 1.00
Epoch: 5, Loss: 0.564, Accuracy: 1.00


In [88]:
model_scripted = torch.jit.script(model)
model_scripted.save('/Users/kirillkotrikov/Documents/End to end project/end_to_end_project/wine_model_scripted.pt')

In [89]:
model_loaded = torch.jit.load('/Users/kirillkotrikov/Documents/End to end project/end_to_end_project/wine_model_scripted.pt')
model_loaded.eval()

RecursiveScriptModule(
  original_name=Sequential
  (0): RecursiveScriptModule(original_name=Linear)
  (1): RecursiveScriptModule(original_name=Softmax)
)

In [91]:
torch.argmax(model_loaded(torch.tensor([[1.5, 1.5]], dtype=torch.float32)), dim=1).item()

1